In [302]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
from functools import reduce
import pandas as pd
import pickle
from typing import List

In [303]:
spark = SparkSession.builder.appName('Exploracao').getOrCreate()

# Comentários

In [304]:
lista_assunto = [
    'power_bi',
    'python_and_dados',
    'cities_skylines',
    'cities_skylines_2',
    'linux',
    'linux_gamming',
    'genshin_impact',
    'zelda'
]

datas = [
    'extracao_data_2024_01_17',
    'extracao_data_2024_01_18',
    'extracao_data_2024_01_19',
    'extracao_data_2024_01_20',
    'extracao_data_2024_01_21',
    'extracao_data_2024_01_22',
    'extracao_data_2024_01_23',
]

In [305]:
df_comentarios = spark.read.json(f'../dados/bronze/assunto_power_bi/extracao_data_2024_01_17/comentarios/req_comentarios.json')

In [306]:
df_comentarios.printSchema()

root
 |-- data_extracao: string (nullable = true)
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- canReply: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- isPublic: boolean (nullable = true)
 |    |    |    |-- topLevelComment: struct (nullable = true)
 |    |    |    |    |-- etag: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- kind: string (nullable = true)
 |    |    |    |    |-- snippet: struct (nullable = true)
 |    |    |    |    |    |-- authorChannelId: struct (nullable = true)
 |    |    |    |    |    |    |-- value: string (nullable = true)
 |    |    |    |    |    |-- authorChannelUrl: str

In [307]:
def abrir_comentarios( lista_datas: List, lista_assunto: List) -> List[DataFrame]:
    lista_df = []
    for assunto in lista_assunto:
        for datas in lista_datas:
            df_comentarios = spark.read.json(f'../dados/bronze/assunto_{assunto}/{datas}/comentarios/req_comentarios.json')
            
            dataframe = df_comentarios.select(
                    f.explode('items').alias('item'),
                    'data_extracao',
                    'etag',
                    'kind',
                    'pageInfo',
                    
                ).select(
                    'item.id',
                    'item.snippet.channelId',
                    'item.snippet.videoId',
                    'item.snippet.topLevelComment.snippet.textOriginal',
                    
                
            )
            lista_df.append(dataframe)
    return lista_df


In [308]:
lista_df = abrir_comentarios(lista_datas=datas, lista_assunto=lista_assunto)

In [309]:
df_final = reduce(lambda df1, df2: df1.union(df2), lista_df)
df_final.show(truncate=False)

24/02/29 22:01:35 WARN DAGScheduler: Broadcasting large task binary with size 1956.2 KiB
+--------------------------+------------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                        |channelId               |videoId    |textOriginal                                                                                            

In [310]:
df_comentarios_pandas = df_final.toPandas()
df_comentarios_pandas.head()

24/02/29 22:01:41 WARN DAGScheduler: Broadcasting large task binary with size 1943.6 KiB


,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise\nLIKE"
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...


In [311]:
df_comentarios_pandas[['id', 'channelId', 'videoId', 'textOriginal']] = df_comentarios_pandas[['id', 'channelId', 'videoId', 'textOriginal']].astype('string') 

In [312]:
df_comentarios_pandas = df_comentarios_pandas.drop_duplicates()

# Resposta Comentários

In [313]:
def abrir_respostas_comentarios( lista_datas: List, lista_assunto: List) -> List[DataFrame]:
    lista_df = []
    for assunto in lista_assunto:
        for datas in lista_datas:
            df_resposta_comentarios = spark.read.json(f'../dados/bronze/assunto_{assunto}/{datas}/resposta_comentarios/resposta_comentarios.json')
            
            dataframe = df_resposta_comentarios.select(
                f.explode('items').alias('item'),
                            
            ).select(
                'item.id',
                'item.snippet.channelId',
                'item.snippet.textOriginal',
                'item.snippet.parentId'
            )
            lista_df.append(dataframe)
    return lista_df

In [314]:
lista_df_resposta_comentarios = abrir_respostas_comentarios(lista_assunto=lista_assunto, lista_datas=datas)

In [315]:
df_resposta_comentarios_final = reduce(lambda df1, df2: df1.union(df2), lista_df_resposta_comentarios)
df_resposta_comentarios_final.show(truncate=False)

24/02/29 22:01:56 WARN DAGScheduler: Broadcasting large task binary with size 1210.4 KiB
+-------------------------------------------------+------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------+
|id                                               |channelId               |textOriginal                                                                                                                                                                                                                                                                                                                                             |parentId                  |
+----------

In [316]:
df_resposta_comentarios_final_pandas = df_resposta_comentarios_final.toPandas()

24/02/29 22:01:58 WARN DAGScheduler: Broadcasting large task binary with size 1197.6 KiB


In [317]:
spark.stop()

In [318]:
df_resposta_comentarios_final_pandas[['id', 'channelId', 'textOriginal', 'parentId']] = df_resposta_comentarios_final_pandas[['id', 'channelId', 'textOriginal', 'parentId']].astype('string')

In [319]:
df_resposta_comentarios_final_pandas = df_resposta_comentarios_final_pandas.drop_duplicates()

# Concatenando dataframes de comentários

In [320]:
df_comentarios_pandas.head()

,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise LIKE"
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...


In [321]:
df_resposta_comentarios_final_pandas.head()


,id,channelId,textOriginal,parentId
0,UgzZff_AwK5aYnauP6h4AaABAg.9zg9VNzo-B29zgPKKBrNVO,UCcuLEJUZy4jo9iFvngkoBUg,Há pelo menos dois,UgzZff_AwK5aYnauP6h4AaABAg
1,UgzZff_AwK5aYnauP6h4AaABAg.9zg9VNzo-B29zgQElZCdaG,UCcuLEJUZy4jo9iFvngkoBUg,grato pelo retorno​@@SaladeEmprego,UgzZff_AwK5aYnauP6h4AaABAg
2,UgzZff_AwK5aYnauP6h4AaABAg.9zg9VNzo-B29zgQFicpN39,UCcuLEJUZy4jo9iFvngkoBUg,grato pelo retorno​@@SaladeEmprego,UgzZff_AwK5aYnauP6h4AaABAg
3,UgxIk3_tEF6SSnEjGDV4AaABAg.9zgB2WyYMq69zgPIGQSki6,UCcuLEJUZy4jo9iFvngkoBUg,Sucesso!,UgxIk3_tEF6SSnEjGDV4AaABAg
4,UgzQwjfcCgBDQVEgCkt4AaABAg.9zfoXBRb8Lb9zgBDjFoOcm,UCcuLEJUZy4jo9iFvngkoBUg,Normalmente a entrevista em grupo costuma ser ...,UgzQwjfcCgBDQVEgCkt4AaABAg


In [322]:
df_comentarios_completos = pd.merge(
    left=df_comentarios_pandas, 
    right=df_resposta_comentarios_final_pandas,
    how='left',
    left_on='id',
    right_on='parentId',

)

In [323]:
df_comentarios_completos.head()

,id_x,channelId_x,videoId,textOriginal_x,id_y,channelId_y,textOriginal_y,parentId
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.,UgwBml2VBGvh47H6CgZ4AaABAg.9zfMad51A7T9zfT8vqYrxG,UCENbpaPgVhO2GtlNcwAJ8mA,"Obrigado. Hoje, quarta dia 17 às 20h, é a Mast...",UgwBml2VBGvh47H6CgZ4AaABAg
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise LIKE",UgzOiQoDWs3JCBPLywx4AaABAg.9zgaq6XZ-0X9ziR1uLspMi,UCnTTl5-22qOKRSVmuGwm1Lw,Valeu 😀,UgzOiQoDWs3JCBPLywx4AaABAg
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra,<NA>,<NA>,<NA>,<NA>
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11,Ugze4k37m59w4PSiHzp4AaABAg.9zfem8NWX6O9zfhNt0aRfp,UCnTTl5-22qOKRSVmuGwm1Lw,"boa, mais um infra pra gente falar aqui no can...",Ugze4k37m59w4PSiHzp4AaABAg
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...,UgyDJGKKqh8emjbSF054AaABAg.9zfasc_aByl9zfbBVmB5Xx,UCnTTl5-22qOKRSVmuGwm1Lw,isso faz com que o Patrimônio do fundo fique p...,UgyDJGKKqh8emjbSF054AaABAg


In [324]:
df_comentarios_completos.shape

(14348, 8)

In [325]:
df_comentarios_completos_um = df_comentarios_completos[['id_x', 'channelId_x', 'videoId', 'textOriginal_x']]
df_comentarios_completos_um.rename(columns={
    'id_x': 'id',
    'channelId_x': 'channelId',
    'textOriginal_x': 'textOriginal'
}, inplace=True)
df_comentarios_completos_um

/tmp/ipykernel_4821/1939900459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comentarios_completos_um.rename(columns={


,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise LIKE"
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...
...,...,...,...,...
14343,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,HAHAHAHAHAHAHAHAHAHA mano o Hélder gritando e ...
14344,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,HAHAHAHAHAHAHAHAHAHA mano o Hélder gritando e ...
14345,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,HAHAHAHAHAHAHAHAHAHA mano o Hélder gritando e ...
14346,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,HAHAHAHAHAHAHAHAHAHA mano o Hélder gritando e ...


In [326]:
df_comentarios_completos_dois = df_comentarios_completos[['id_x', 'channelId_x', 'videoId', 'textOriginal_y']]
df_comentarios_completos_dois.rename(columns={
    'id_x': 'id',
    'channelId_x': 'channelId',
    'textOriginal_y': 'textOriginal'
}, inplace=True)
df_comentarios_completos_dois

/tmp/ipykernel_4821/112964936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comentarios_completos_dois.rename(columns={


,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,"Obrigado. Hoje, quarta dia 17 às 20h, é a Mast..."
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Valeu 😀
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,<NA>
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"boa, mais um infra pra gente falar aqui no can..."
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,isso faz com que o Patrimônio do fundo fique p...
...,...,...,...,...
14343,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,😅
14344,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,eu achava que era edição do Bezzi pra não assu...
14345,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,O Hélder te pega demais?
14346,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,"Ah. ...pode crer ... hahahaha, deve ser um sis..."


In [327]:
df_comentarios_completo = pd.concat([df_comentarios_completos_um, df_comentarios_completos_dois])
df_comentarios_completo

,id,channelId,videoId,textOriginal
0,UgwBml2VBGvh47H6CgZ4AaABAg,UCENbpaPgVhO2GtlNcwAJ8mA,nDfmW0LMAto,Excelente.
1,UgzOiQoDWs3JCBPLywx4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,"Valeu, ótima análise LIKE"
2,UgwcYJjgmACxqvHtGQl4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Já tenho e gosto desse fi infra
3,Ugze4k37m59w4PSiHzp4AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,OGIN11
4,UgyDJGKKqh8emjbSF054AaABAg,UCnTTl5-22qOKRSVmuGwm1Lw,_rP6fm9avmQ,Não achei nada interessante essa questão de nã...
...,...,...,...,...
14343,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,😅
14344,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,eu achava que era edição do Bezzi pra não assu...
14345,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,O Hélder te pega demais?
14346,UgydPIbSBxU804gC97N4AaABAg,UC2bZgihqibFD_vhaYEXQZFg,UNSvXnoM2PA,"Ah. ...pode crer ... hahahaha, deve ser um sis..."


# Abrindo dataframe de canal, vídeo

In [328]:
df_assunto = pd.read_parquet('../dados/ouro/dados_tratado_estatisticas_gerais.parquet', columns=['ASSUNTO', 'ID_CANAL', 'NM_CANAL', 'ID_VIDEO', 'TITULO_VIDEO'])
df_assunto

,ASSUNTO,ID_CANAL,NM_CANAL,ID_VIDEO,TITULO_VIDEO
0,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts
1,assunto_python_and_dados,UC70kawNmZCDeZDZBbeiolHA,Wesley Roberto | Data Analytics,S8FcfOPjFJY,Como se tornar um Analista de dados em 2024
2,assunto_python_and_dados,UCiYrw6iAqlYXvP7no7h4tUQ,Rodolfo De Nadai,s9hUoajuWwc,Python: Structure Data Types
3,assunto_python_and_dados,UCNblNh-u0nu5mtXT84Wgdiw,Mestres da Informática,j1-IbFM7TtE,🔴O Novo Ciclo - 1° Fase | O Detetive Virtual
4,assunto_python_and_dados,UCU5vSFBBO_cp6ie0OG74b2g,Refatorando,5Rl2CnW5ZFc,Como é o dia a dia de trabalho de um Programador
...,...,...,...,...,...
12063,assunto_cities_skylines_2,UCGved3ynk6ZhUvSKxzgweTw,Jogando,tjZZTkpmPo4,"07 Itaquera - Cities Skylines 2 UNIVERSIDADE, ..."
12064,assunto_cities_skylines_2,UCdrd9d0-1p8sgXOvJDc-naA,FLAVIOSUICO GAMES,J0O_HqOo4UE,Cities Skyline 2 - INTERSECÇÃO ENORME - Gamepl...
12065,assunto_cities_skylines_2,UCDqzBrUOm7guqqyU8eDackg,BUG CITIES,AHKQdCqYoHI,Cidade (Mods) CITIES SKYLINES I (iniciante)| C...
12066,assunto_cities_skylines_2,UCDqzBrUOm7guqqyU8eDackg,BUG CITIES,AHKQdCqYoHI,Cidade (Mods) CITIES SKYLINES I (iniciante)| C...


In [329]:
df_assunto = df_assunto.drop_duplicates()

In [330]:
df_comentarios_completo_assunto = pd.merge(
    left=df_assunto,
    right=df_comentarios_completo,
    how='inner',
    left_on='ID_CANAL',
    right_on='channelId'
)
df_comentarios_completo_assunto

,ASSUNTO,ID_CANAL,NM_CANAL,ID_VIDEO,TITULO_VIDEO,id,channelId,videoId,textOriginal
0,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts,UgzKXZmYCVUilpkbt9x4AaABAg,UCMqGy4xIIGs01ZVcBv0B8Cw,k9d_QvCQjaI,Cadê o vídeo de reportlab mano João?
1,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts,UgwS_UPA2qZHH53zxaZ4AaABAg,UCMqGy4xIIGs01ZVcBv0B8Cw,12PLJksc12k,Legal ❤
2,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts,UgyywtKi1HSwTpz8H2Z4AaABAg,UCMqGy4xIIGs01ZVcBv0B8Cw,12PLJksc12k,"Muito, bom continue fazendo mais...👏👏👏"
3,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts,UgzKXZmYCVUilpkbt9x4AaABAg,UCMqGy4xIIGs01ZVcBv0B8Cw,k9d_QvCQjaI,"Estou com problemas tecnicos, Laptop nao esta ..."
4,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts,UgwS_UPA2qZHH53zxaZ4AaABAg,UCMqGy4xIIGs01ZVcBv0B8Cw,12PLJksc12k,<NA>
...,...,...,...,...,...,...,...,...,...
327551,assunto_cities_skylines_2,UCHJJ9aeqrnliy6xDTqTWz_g,Boring Bones Tv,gvrvx8pCh4M,O CRIAR UM SIM DO THE SIMS 2 BETA ERA INCRÍVEL...,Ugxs80FCxPQdfRog7Xh4AaABAg,UCHJJ9aeqrnliy6xDTqTWz_g,gvrvx8pCh4M,<NA>
327552,assunto_cities_skylines_2,UCHJJ9aeqrnliy6xDTqTWz_g,Boring Bones Tv,gvrvx8pCh4M,O CRIAR UM SIM DO THE SIMS 2 BETA ERA INCRÍVEL...,Ugy0m_IlxXymyvDxwVh4AaABAg,UCHJJ9aeqrnliy6xDTqTWz_g,gvrvx8pCh4M,"Nao brincaaaaa, serioooo? :o Caracaaaa eu to c..."
327553,assunto_cities_skylines_2,UCHJJ9aeqrnliy6xDTqTWz_g,Boring Bones Tv,gvrvx8pCh4M,O CRIAR UM SIM DO THE SIMS 2 BETA ERA INCRÍVEL...,Ugz0tT97md19_R-yfgV4AaABAg,UCHJJ9aeqrnliy6xDTqTWz_g,gvrvx8pCh4M,Surra mtooooooo Hahahah e isso é de 21 anos at...
327554,assunto_cities_skylines_2,UCHJJ9aeqrnliy6xDTqTWz_g,Boring Bones Tv,gvrvx8pCh4M,O CRIAR UM SIM DO THE SIMS 2 BETA ERA INCRÍVEL...,Ugz0tT97md19_R-yfgV4AaABAg,UCHJJ9aeqrnliy6xDTqTWz_g,gvrvx8pCh4M,"Tudo menos o cas, o cas de the Sims 4 é incrível."


In [331]:
df_comentarios_completo_assunto = pd.merge(
    left=df_assunto,
    right=df_comentarios_completo,
    how='inner',
    left_on='ID_CANAL',
    right_on='channelId'
)
df_comentarios_completo_assunto.drop(['channelId'], axis=1, inplace=True)
df_comentarios_completo_assunto.head()

,ASSUNTO,ID_CANAL,NM_CANAL,ID_VIDEO,TITULO_VIDEO,id,videoId,textOriginal
0,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts,UgzKXZmYCVUilpkbt9x4AaABAg,k9d_QvCQjaI,Cadê o vídeo de reportlab mano João?
1,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts,UgwS_UPA2qZHH53zxaZ4AaABAg,12PLJksc12k,Legal ❤
2,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts,UgyywtKi1HSwTpz8H2Z4AaABAg,12PLJksc12k,"Muito, bom continue fazendo mais...👏👏👏"
3,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts,UgzKXZmYCVUilpkbt9x4AaABAg,k9d_QvCQjaI,"Estou com problemas tecnicos, Laptop nao esta ..."
4,assunto_python_and_dados,UCMqGy4xIIGs01ZVcBv0B8Cw,Usando Python,BVHN3nK8JR8,Conversão para String (str) em Python.#shorts,UgwS_UPA2qZHH53zxaZ4AaABAg,12PLJksc12k,<NA>


In [332]:
df_comentarios_completo_assunto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327556 entries, 0 to 327555
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ASSUNTO       327556 non-null  string
 1   ID_CANAL      327556 non-null  string
 2   NM_CANAL      327556 non-null  string
 3   ID_VIDEO      327556 non-null  string
 4   TITULO_VIDEO  327556 non-null  string
 5   id            327556 non-null  string
 6   videoId       327556 non-null  string
 7   textOriginal  240094 non-null  string
dtypes: string(8)
memory usage: 20.0 MB


In [333]:
df_comentarios_completo_assunto.to_parquet('../dados/ouro/comentarios.parquet')